In [1]:
import re
import json

# with open('./ss-ner_sampled.json') as f:
#     data = json.load(f)
    
# tokenized = re.split(r'\s+', data["body"])
# entities = data["entities"]
# print(data)


In [2]:
with open('./tag_to_id.json') as f:
    types = json.load(f)

In [3]:
def convert_to_indexed_tags(data):
    pattern = r"\s+" # need to split any number of whitespace
    tokenized = re.split(pattern, data["body"])
    entities = data["entities"]
    cur_start = 0
    cur_end = 0
    state = "O" # Outside
    tags = []
    i = 0
    for token in tokenized:
        cur_end = cur_start + len(token) # token lies in [cur_start,cur_end], the extra 1 comes from space
    #     print(data['body'][cur_start:cur_end])
    #     print(cur_end)

        if state == "O" and cur_start < entities[i]['end'] <= cur_end:
            tags.append("B-" + str(entities[i]['type']))
            state = "I-" + str(entities[i]['type'])
        elif state.startswith("I-") and cur_start < entities[i]['start'] <= cur_end:
            tags.append(state)
            state = "O"
            i+=1
        elif cur_start > entities[i]['end']:
            tags.append(state)
            state = "O"
            i+=1
        else:
            tags.append(state)

        if(i >= len(entities)): break # we have iterated all the possible entity matches

        cur_start = cur_end + 1

        while(cur_start < len(data['body']) and data['body'][cur_start] == ' '): # skip trailing or leading whitespace
#             output.append("O")
            cur_start += 1
        
    
#     print(tags)
    
    for i in range(len(tags)):
        for key, value in types.items():
            if(tags[i].lower() == key.lower()):
                tags[i] = value 
                break
    return tags, tokenized


In [4]:
print(types)

{'O': 0, 'B-country/region': 1, 'I-country/region': 2, 'B-coronavirus': 3, 'I-coronavirus': 4, 'B-livestock': 5, 'I-livestock': 6, 'B-wildlife': 7, 'I-wildlife': 8, 'B-evolution': 10, 'I-evolution': 9, 'B-physical_science': 11, 'I-physical_science': 12, 'B-substrate': 13, 'I-substrate': 14, 'B-material': 15, 'I-material': 16, 'B-immune_response': 17, 'I-immune_response': 18, 'B-covid_activities': 19, 'I-covid_activities': 20}


In [5]:
tokenized_articles = []
converted_tags = []
with open('./SS-NER-full.json') as f:
    for article in f:
        datum = json.loads(article)
        tag, tokenized = convert_to_indexed_tags(datum)
        tokenized_articles.append(tokenized)
        converted_tags.append(tag)

# print(data)
# print(tags)

In [6]:
# print(tokenized_articles[1])
# print(converted_tags[1])

In [7]:
def merge_dots(words, converted_tag, start):
    str_words = [] # same sentence
    str_words_tags = []
    end = start
    while(end < len(words)-1 and (len(words[end]) == 0 or words[end][-1] != '.')):
        str_words.append(words[end])
        str_words_tags.append(int(converted_tag[end])) # index must be of number type
        end+=1
    str_words.append(words[end]) # add the final word that contains dot
    str_words_tags.append(int(converted_tag[end]))
    end += 1
    return str_words, str_words_tags, end # end would be used in the next iteration

In [8]:
def split_article_by_dot(tokenized_article, converted_tag, output):
    next_start = 0
#     sentences = []
#     sentences_tags = []
#     word_obj_arr = []   
    while(next_start < len(tokenized_article)):
        str_words_arr, str_words_tags_arr, next_start = merge_dots(tokenized_article, converted_tag, next_start)
        word_obj = {"str_words":str_words_arr, "tags":str_words_tags_arr}
        output.append(word_obj)
        
#     return word_obj_arr

In [9]:
output = []
for i in range(len(tokenized_articles)):
# for i in range(1):
    try:
        split_article_by_dot(tokenized_articles[i], converted_tags[i], output)
#         output.extend(article_word_obj_arr) # add all elements of the list to output
    except:
        pass # we don't care about the errors
# print(converted_tags[0])


In [10]:
# with open('data.json', 'w') as f:
#     json.dump(output, f)

In [11]:
# print(len(tokenized_articles))


traingin_set = []
for i in range(dev_end,len(output)-int(len(output) * 0.94)):
    traingin_set.append(output[i])

In [ ]:
test_set = []
test_end = int(len(output) * 0.01)
for i in range(test_end):
    test_set.append(output[i])
    
dev_set = []
dev_end = int(len(output) * 0.01) + test_end
for i in range(test_end,dev_end):
    dev_set.append(output[i])

In [12]:
with open('test.json', 'w') as f:
    json.dump(test_set , f)

In [13]:
with open('dev.json', 'w') as f:
    json.dump(dev_set , f)

In [14]:
with open('train.json', 'w') as f:
    json.dump(traingin_set , f)